In [1]:
import json
import typing
from base64 import b64encode, b64decode
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from PIL import Image
import io

In [2]:
def encrypt_AES(data:bytes)->typing.Tuple[bytes, str]:
    key = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_CTR)
    ct_bytes = cipher.encrypt(data)
    nonce = b64encode(cipher.nonce).decode('utf-8')
    ct = b64encode(ct_bytes).decode('utf-8')
    result = json.dumps({'nonce':nonce, 'ciphertext':ct})
    return key, result

In [3]:
def decrypt_AES(key:bytes, json_input:str)->bytes:
    # We assume that the key was securely shared beforehand
    try:
        b64 = json.loads(json_input)
        nonce = b64decode(b64['nonce'])
        ct = b64decode(b64['ciphertext'])
        cipher = AES.new(key, AES.MODE_CTR, nonce=nonce)
        pt = cipher.decrypt(ct)
        return pt
    except (ValueError, KeyError):
        print("Incorrect decryption")

In [4]:
with open('./assets/img/cat.jpg', 'rb') as f:
    data = f.read()
key, encrypted_json = encrypt_AES(data)
key, len(encrypted_json)

(b'{\x02`8<\xeciQ\x98\xecF\xffa\xach\xed', 25923)

In [5]:
pt = decrypt_AES(key, encrypted_json)
img = Image.open(io.BytesIO(pt))
img.show()